In [42]:
import numpy as np
from time import clock
from __future__ import print_function
import tensorflow as tf

In [49]:
class Network():

    alpha = 0.1

    def __init__(self, learning_rate=0.1):
        ''' initialize the classifier with default (best) parameters '''
        # TODO
        self.alpha = learning_rate

        # Parameters
        learning_rate = 0.001
        self.batch_size = 30
        self.display_step = 1

        # Network Parameters
        n_hidden_1 = 256 # 1st layer number of neurons
        n_hidden_2 = 256 # 2nd layer number of neurons
        n_input = 294 # Data input (line shape: 294)
        n_classes = 6 # Classes (0-5 digits)

        # tf Graph input
        self.X = tf.placeholder("float", [None, n_input])
        self.Y = tf.placeholder("float", [None, n_classes])

        # Store layers weight & bias
        weights = {
            'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
            'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
            'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
        }
        biases = {
            'b1': tf.Variable(tf.random_normal([n_hidden_1])),
            'b2': tf.Variable(tf.random_normal([n_hidden_2])),
            'out': tf.Variable(tf.random_normal([n_classes]))
        }

        # Hidden fully connected layer with 256 neurons
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(self.X, weights['h1']), biases['b1']))
        # Hidden fully connected layer with 256 neurons
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
        # Output fully connected layer with a neuron for each class
        self.logits = tf.matmul(layer_2, weights['out']) + biases['out']

        # Define loss and optimizer
        self.loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.Y))
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        self.train_op = optimizer.minimize(self.loss_op)

        # Initializing the variables
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)

    def fit(self,X,Y,warm_start=True,n_epochs=1000):
        ''' train the network, and if warm_start, then do not reinit. the network
            (if it has already been initialized)
        '''
        train_num_examples = Y.shape[0]

        # Training cycle
        for epoch in range(n_epochs):
            avg_cost = 0.
            total_batch = int(train_num_examples/self.batch_size)
            current_batch_index = 0

            # Loop over all batches
            for i in range(total_batch):
                next_batch_index = current_batch_index+self.batch_size
                batch_x = X[current_batch_index:next_batch_index,:]
                batch_y = Y[current_batch_index:next_batch_index,:]
                current_batch_index = next_batch_index

                # Run optimization op (backprop) and cost op (to get loss value)
                _, c = self.sess.run([self.train_op, self.loss_op], feed_dict={self.X: batch_x, self.Y: batch_y})
                # Compute average loss
                avg_cost += c / total_batch
            # Display logs per epoch step
            if epoch % self.display_step == 0:
                print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
        print("Optimization Finished!")

    def predict_proba(self,X):
        ''' return a matrix P where P[i,j] = P(Y[i,j]=1), 
        for all instances i, and labels j. '''

        predictions = {
              # Generate predictions (for PREDICT and EVAL mode)
              "classes": tf.argmax(input=self.logits, axis=1),
              # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
              # `logging_hook`.
              "probabilities": tf.nn.softmax(self.logits, name="softmax_tensor")
        }

        return predictions["probabilities"].eval(feed_dict={self.X: X}, session=self.sess)

    def predict(self,X):
        ''' return a matrix of predictions for X '''
        return (self.predict_proba(X) >= 0.5).astype(int)

In [50]:
###############################################################
# This is an example script, you may modify it as you wish
###############################################################

# Load and parse the data (N instances, D features, L=6 labels)
XY = np.genfromtxt('data/scene.csv', skip_header=1, delimiter=",")
N,DL = XY.shape
L = 6
D = DL - L
Y = XY[:,0:L].astype(int)
X = XY[:,L:D+L]

# Split into train/test sets
n = int(N*6/10)
X_train = X[0:n]
Y_train = Y[0:n]
X_test = X[n:]
Y_test = Y[n:]

t0 = clock()

In [51]:
h = Network()

In [52]:
h.fit(X_train,Y_train)

Epoch: 0001 cost=4.621461561
Epoch: 0002 cost=2.083514309
Epoch: 0003 cost=1.562849268
Epoch: 0004 cost=1.293149619
Epoch: 0005 cost=1.106317174
Epoch: 0006 cost=0.966146655
Epoch: 0007 cost=0.871938796
Epoch: 0008 cost=0.793317725
Epoch: 0009 cost=0.731915019
Epoch: 0010 cost=0.679200709
Epoch: 0011 cost=0.634721684
Epoch: 0012 cost=0.594364538
Epoch: 0013 cost=0.558870896
Epoch: 0014 cost=0.530507234
Epoch: 0015 cost=0.506102574
Epoch: 0016 cost=0.478192138
Epoch: 0017 cost=0.456212284
Epoch: 0018 cost=0.437316229
Epoch: 0019 cost=0.420225370
Epoch: 0020 cost=0.401812959
Epoch: 0021 cost=0.393969361
Epoch: 0022 cost=0.396805567
Epoch: 0023 cost=0.387364576
Epoch: 0024 cost=0.374652082
Epoch: 0025 cost=0.364335100
Epoch: 0026 cost=0.350472885
Epoch: 0027 cost=0.335761973
Epoch: 0028 cost=0.327285542
Epoch: 0029 cost=0.320510723
Epoch: 0030 cost=0.311554708
Epoch: 0031 cost=0.304227384
Epoch: 0032 cost=0.299973099
Epoch: 0033 cost=0.297759534
Epoch: 0034 cost=0.295465402
Epoch: 0035 co

Epoch: 0287 cost=0.360481730
Epoch: 0288 cost=0.365770544
Epoch: 0289 cost=0.370422234
Epoch: 0290 cost=0.374500483
Epoch: 0291 cost=0.379536537
Epoch: 0292 cost=0.386848775
Epoch: 0293 cost=0.389883474
Epoch: 0294 cost=0.396187571
Epoch: 0295 cost=0.400707357
Epoch: 0296 cost=0.410883035
Epoch: 0297 cost=0.416879322
Epoch: 0298 cost=0.418328975
Epoch: 0299 cost=0.420011337
Epoch: 0300 cost=0.416351527
Epoch: 0301 cost=0.415208076
Epoch: 0302 cost=0.438015415
Epoch: 0303 cost=0.582978412
Epoch: 0304 cost=0.590319350
Epoch: 0305 cost=0.401369604
Epoch: 0306 cost=0.381681613
Epoch: 0307 cost=0.373058443
Epoch: 0308 cost=0.376895139
Epoch: 0309 cost=0.377195037
Epoch: 0310 cost=0.385339806
Epoch: 0311 cost=0.390122476
Epoch: 0312 cost=0.395670266
Epoch: 0313 cost=0.400369139
Epoch: 0314 cost=0.405432173
Epoch: 0315 cost=0.410181584
Epoch: 0316 cost=0.414170342
Epoch: 0317 cost=0.418680399
Epoch: 0318 cost=0.423731309
Epoch: 0319 cost=0.428113093
Epoch: 0320 cost=0.432927716
Epoch: 0321 co

Epoch: 0572 cost=0.782298553
Epoch: 0573 cost=0.784731264
Epoch: 0574 cost=0.786977259
Epoch: 0575 cost=0.789104533
Epoch: 0576 cost=0.791204044
Epoch: 0577 cost=0.793240501
Epoch: 0578 cost=0.795245598
Epoch: 0579 cost=0.797184601
Epoch: 0580 cost=0.799153839
Epoch: 0581 cost=0.801276065
Epoch: 0582 cost=0.803324891
Epoch: 0583 cost=0.805125950
Epoch: 0584 cost=0.806785329
Epoch: 0585 cost=0.807607550
Epoch: 0586 cost=0.809043044
Epoch: 0587 cost=0.810661337
Epoch: 0588 cost=0.812529670
Epoch: 0589 cost=0.814322981
Epoch: 0590 cost=0.816302790
Epoch: 0591 cost=0.818441547
Epoch: 0592 cost=0.821153058
Epoch: 0593 cost=0.824672499
Epoch: 0594 cost=0.829272046
Epoch: 0595 cost=0.833187530
Epoch: 0596 cost=0.835361579
Epoch: 0597 cost=0.837306271
Epoch: 0598 cost=0.838866667
Epoch: 0599 cost=0.840949201
Epoch: 0600 cost=0.842260351
Epoch: 0601 cost=0.843735240
Epoch: 0602 cost=0.846055552
Epoch: 0603 cost=0.848159746
Epoch: 0604 cost=0.850533760
Epoch: 0605 cost=0.852333424
Epoch: 0606 co

Epoch: 0855 cost=1.241792186
Epoch: 0856 cost=1.246475973
Epoch: 0857 cost=1.251165294
Epoch: 0858 cost=1.256257788
Epoch: 0859 cost=1.261900622
Epoch: 0860 cost=1.268034237
Epoch: 0861 cost=1.274675318
Epoch: 0862 cost=1.282858601
Epoch: 0863 cost=1.291905375
Epoch: 0864 cost=1.298010985
Epoch: 0865 cost=1.302693386
Epoch: 0866 cost=1.307043729
Epoch: 0867 cost=1.310971777
Epoch: 0868 cost=1.312638929
Epoch: 0869 cost=1.317988111
Epoch: 0870 cost=1.324844653
Epoch: 0871 cost=1.334284044
Epoch: 0872 cost=1.333150578
Epoch: 0873 cost=1.335482646
Epoch: 0874 cost=1.338283254
Epoch: 0875 cost=1.341956944
Epoch: 0876 cost=1.346532372
Epoch: 0877 cost=1.351162334
Epoch: 0878 cost=1.355686630
Epoch: 0879 cost=1.360443248
Epoch: 0880 cost=1.365854253
Epoch: 0881 cost=1.371909911
Epoch: 0882 cost=1.379165791
Epoch: 0883 cost=1.386649465
Epoch: 0884 cost=1.393500046
Epoch: 0885 cost=1.401094369
Epoch: 0886 cost=1.407819202
Epoch: 0887 cost=1.413765273
Epoch: 0888 cost=1.418796943
Epoch: 0889 co

In [53]:
proba = h.predict_proba(X_test)

In [54]:
Y_pred = h.predict(X_test)
Y_pred

array([[0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0]])

In [55]:
loss = np.mean(Y_pred != Y_test)
print("Hamming loss: ", loss)

Hamming loss:  0.12729166666666666
